# Intro

This assumes you have gone through the intro/mpi4py intro, and that you are comfortable generating ExaGO output.

Now we will go through specifics to the vizualisation frontend, how it is installed, and a typical workflow.

# Open Issues:

- npm dependencies were pinned and have security issues
- requirements.txt has pinned versions and we cannot upgrade / reproduce
- OpenAI keys and other missing steps...

# Generating file to visualize

I don't know how to do gic file option, so we are going to use OPFLOW C++ CLI to do this...

In [8]:
!opflow --version

ExaGO 1.6.0 built on Nov  9 2023


TODO: Going to let Shri fill the details in here, as likely we don't want people churning large data sets in a small demo, and we can include pre-generated data I would hope...

In [ ]:
!./opflow -netfile <netfile> -save_output -opflow_output_format JSON -gicfile <gicfilename>  

# Visualization

Launch through shell commands or with the following shell command:

In [1]:
!cd /home/app/viz && . /usr/local/share/nvm/nvm.sh && nvm install 16.13.0 && nvm use 16.13.0 && node --version && npm install && npm start

v16.13.0 is already installed.
Now using node v16.13.0 (npm v8.1.0)
Now using node v16.13.0 (npm v8.1.0)
v16.13.0
npm WARN EBADENGINE Unsupported engine {ompleted in 1ms[K
npm WARN EBADENGINE   package: '@fortaine/fetch-event-source@3.0.6',
npm WARN EBADENGINE   required: { node: '>=16.15' },
npm WARN EBADENGINE   current: { node: 'v16.13.0', npm: '8.1.0' }
npm WARN EBADENGINE }
npm WARN EBADENGINE Unsupported engine {
npm WARN EBADENGINE   package: 'langchain@0.0.75',
npm WARN EBADENGINE   required: { node: '>=18' },
npm WARN EBADENGINE   current: { node: 'v16.13.0', npm: '8.1.0' }
npm WARN EBADENGINE }
m##################⸩ ⠴ reify:fsevents: timing reifyNode:node_modules/webpack-des0ms
up to date, audited 1436 packages in 5s

91 packages are looking for funding
  run `npm fund` for details

16 vulnerabilities (1 low, 6 moderate, 9 high)

To address issues that do not require attention, run:
  npm audit fix

To address all issues (including breaking changes), run:
  npm audit fix --for

I seem to be stuck... Let's try and capture some more debugging information really quick.

# ChatGrid

In [2]:
!cd ../../viz/backend && \
    sudo python3 -m pip install --upgrade pip && \
    sudo python3 -m pip install -r requirements.txt && \
    PYTHONPATH=/usr/local/lib/python3.11/site-packages python3 server.py

 * Serving Flask app 'server'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 103-781-530
127.0.0.1 - - [10/Nov/2023 04:03:42] "GET /?vscodeBrowserReqId=1699589022090 HTTP/1.1" 404 -


: 

It seems like there were not only windows file endings here, but there were hard coded frozen requirements that were just for windows, and pinned for python < 3.11... Trying to run anyway just to see what happens.

Stopping here, where a few dependencies where missing, and PYTHONPATH wasn't quite correct. I am missing quite a few steps, but hopefully this is close...

In [ ]:
!cd ../../viz/backend && python3 server.py


# Installation

This is done through the `postCreateCommand` that is configured in `.devcontainer/devcontainer.json` in theroy. Still debugging, so I did some stuff through Jupyter magic just to iterate quickly.

In [2]:
!cat ../../.devcontainer/devcontainer.json

{
	"build": {
		"dockerfile": "Dockerfile",
		"context": ".."
	},
	"customizations": {
		"vscode": {
			"extensions": [
				"ms-azuretools.vscode-docker",
				"github.vscode-github-actions",
				"ms-toolsai.jupyter",
				"GitHub.copilot",
				"ms-vsliveshare.vsliveshare"
			]
		}
	},
	"workspaceFolder": "/home/app/",
  	"postCreateCommand": "cd /home/app/viz && export CXX=$(which gcc) && npm install",
    "workspaceMount": "source=${localWorkspaceFolder},target=/home/app/,type=bind,consistency=cached"
}


In [3]:
!npm --version

9.2.0


In [5]:
!node --version

v18.13.0


In [7]:
!cat ../../.devcontainer/Dockerfile

# Build stage with Spack pre-installed and ready to be used
FROM spack/ubuntu-jammy as builder

# Install OS packages needed to build the software
RUN apt-get -yqq update && apt-get -yqq upgrade \
 && apt-get -yqq install autoconf \
 && rm -rf /var/lib/apt/lists/*

# What we want to install and how we want to install it
# is specified in a manifest file (spack.yaml)
RUN mkdir /opt/spack-environment \
&&  (echo spack: \
&&   echo '  specs:' \
&&   echo '  - exago@1.6.0+ipopt+python+mpi' \
&&   echo '  - petsc~fortran~hdf5~hypre+metis' \
&&   echo '  - ipopt@3.12.10+coinhsl~mumps+metis' \
&&   echo '  - coinhsl@2019.05.21+blas' \
&&   echo '  - py-jupyterlab' \
&&   echo '  - py-ipyparallel' \
&&   echo '  concretizer:' \
&&   echo '    unify: true' \
&&   echo '    reuse: true' \
&&   echo '  mirrors:' \
&&   echo '    spack: https://binaries.spack.io/develop' \
&&   echo '  packages:' \
&&   echo '    all:' \
&&   echo '      providers:' \
&&   echo '        mpi:' \
&&   echo '        